<a href="https://colab.research.google.com/github/csce585-mlsystems/csce585-midi/blob/main/SetupNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# use your drive for storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# clone the repo
!git clone https://github.com/csce585-mlsystems/csce585-midi.git
%cd csce585-midi

Cloning into 'csce585-midi'...
remote: Enumerating objects: 579, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 579 (delta 123), reused 139 (delta 52), pack-reused 354 (from 1)
Receiving objects: 100% (579/579), 2.38 MiB | 8.98 MiB/s, done.
Resolving deltas: 100% (293/293), done.
/content/csce585-midi


In [6]:
# clone the nottingham dataset
%cd data/
!git clone https://github.com/jukedeck/nottingham-dataset.git nottingham-dataset-master
%cd ..

/content/csce585-midi/data
Cloning into 'nottingham-dataset-master'...
remote: Enumerating objects: 3119, done.
remote: Total 3119 (delta 0), reused 0 (delta 0), pack-reused 3119 (from 1)
Receiving objects: 100% (3119/3119), 879.17 KiB | 4.75 MiB/s, done.
Resolving deltas: 100% (1432/1432), done.
/content/csce585-midi


In [7]:
# install dependencies and activate virtual environment
!uv sync
!source .venv/bin/activate

Using CPython 3.11.10
Creating virtual environment at: .venv
Resolved 72 packages in 6ms
Prepared 67 packages in 1m 56s
Installed 67 packages in 1.03s
 + certifi==2025.10.5
 + chardet==5.2.0
 + charset-normalizer==3.4.4
 + contourpy==1.3.3
 + csce585-midi==0.1.0 (from file:///content/csce585-midi)
 + cycler==0.12.1
 + filelock==3.20.0
 + fonttools==4.60.1
 + fsspec==2025.9.0
 + hf-xet==1.1.10
 + huggingface-hub==0.35.3
 + idna==3.11
 + importlib-resources==6.5.2
 + jinja2==3.1.6
 + joblib==1.5.2
 + jsonpickle==4.1.1
 + kiwisolver==1.4.9
 + markupsafe==3.0.3
 + matplotlib==3.10.7
 + miditok==3.0.6.post1
 + mido==1.3.3
 + more-itertools==10.8.0
 + mpmath==1.3.0
 + music21==9.9.1
 + networkx==3.5
 + numpy==1.26.4
 + nvidia-cublas-cu12==12.8.4.1
 + nvidia-cuda-cupti-cu12==12.8.90
 + nvidia-cuda-nvrtc-cu12==12.8.93
 + nvidia-cuda-runtime-cu12==12.8.90
 + nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cufft-cu12==11.3.3.83
 + nvidia-cufile-cu12==1.13.1.3
 + nvidia-curand-cu12==10.3.9.90
 + nvidia-cu

In [10]:
# preprocess with all tokenization types
!source .venv/bin/activate && ./utils/preprocess_all.sh data/nottingham-dataset-master/MIDI

Processing dataset: nottingham-dataset-master
Source Path: data/nottingham-dataset-master/MIDI
Running Naive Preprocessing...
Preprocessing 3089 files from data/nottingham-dataset-master/MIDI...
100% 3089/3089 [00:02<00:00, 1225.76it/s]
Vocab size: 52
Converting notes to integers...
Creating sequences...
Saved 553131 sequences to data/nottingham-dataset-master_naive/sequences.npy
Running Standard MIDITok Preprocessing...
processing 3089 MIDI files from data/nottingham-dataset-master/MIDI
processing files: 100% 3089/3089 [00:12<00:00, 245.57it/s]
Saved sequences to: data/nottingham-dataset-master_miditok/sequences.npy
/content/csce585-midi/utils/preprocess_miditok.py:117: UserWarning: miditok: The `save_params` method had been renamed `save`. It is now depreciated and will be removed in future updates.
  tokenizer.save_params(tokenizer_path)
Saved config to: data/nottingham-dataset-master_miditok/config.json
Running Measure Dataset Preprocessing...
/content/csce585-midi/.venv/lib/python

In [12]:
# train a quick model to make sure everything works
!source .venv/bin/activate && MPLBACKEND=Agg python training/train_generator.py --dataset data/nottingham-dataset-master_naive --model_type lstm --epochs 4 --max_batches 5

using device: cpu
Vocab size: 52, Sequence length: 50, Device: cpu
dataset initialized (fast mode): 27,656,550 samples from 553131 sequences
quick data check: sample_count=100, max_token_id_in_sampled_sequences=51
interpreted vocab_size = 52 (from vocab.json)
embedding check passed: embedding.num_embeddings = 52
Model type: lstm
Model parameters: 941,620

TRAINING CONFIGURATION
Dataset: data/nottingham-dataset-master_naive
Model: lstm
Epochs: 4
Batch size: 32
Batches per epoch: 5
Learning rate: 0.001

Large datasets may take several minutes per epoch
 Watch for periodic progress updates during training


Epoch 1/4 - Training on 5 batches...
  Batch 2/5 (40.0%) | Loss: 3.9045 | Avg Loss: 3.9216 | Speed: 0.4 batch/s | ETA: 0.1m
  Batch 3/5 (60.0%) | Loss: 3.8554 | Avg Loss: 3.8996 | Speed: 0.5 batch/s | ETA: 0.1m
  Batch 4/5 (80.0%) | Loss: 3.8200 | Avg Loss: 3.8797 | Speed: 0.7 batch/s | ETA: 0.0m
  Batch 5/5 (100.0%) | Loss: 3.7263 | Avg Loss: 3.8490 | Speed: 0.8 batch/s | ETA: 0.0m

E

In [ ]:
# train a larger model (please view the repo's wiki if you want to use CUDA)
!python  training/train_generator.py  \
--dataset data/nottingham_naive \
--model_type lstm \
--epochs  20  \
--batch_size 128 \
--lr  0.001  \
--hidden_size 512

In [19]:
!source .venv/bin/activate && python generate.py --data_dir data/nottingham-dataset-master_naive \
--model_path models/generators/checkpoints/nottingham-dataset-master_naive/lstm* \
--model_type lstm

Inferred architecture: 2 layers, hidden_size=256
Loaded lstm model with 941,620 parameters
Generated midi saved to outputs/generators/nottingham-dataset-master_naive/midi/generated_20251129_155726.mid
Logged generated MIDI details to logs/generators/nottingham-dataset-master_naive/midi/output_midis.csv
Evaluated and logged MIDI evaluation results.
